In [ ]:
# import packages
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
from sentence_transformers import SentenceTransformer
import chromadb
from datasets import load_dataset
from gpt4all import GPT4All
from pathlib import Path

In [2]:
# Embedding vector
class VectorStore:
    def __init__(self, collection_name):
       # Initialize the embedding model
        self.embedding_model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')
        self.chroma_client = chromadb.Client()
        self.collection = self.chroma_client.create_collection(name=collection_name)

    # Method to populate the vector store with embeddings from a dataset
    def populate_vectors(self, dataset):
        # Select the text columns to concatenate
        # title = dataset['train']['title_cleaned'][:1000]  # Limiting to 100 examples for the demo
        recipe = dataset['train']['recipe_new'][:1000]
        allergy = dataset['train']['allergy_type'][:1000]
        ingredients = dataset['train']['ingredients_alternatives'][:1000]

        # Concatenate the text from both columns
        texts = [f"{rep} {ingr} {alle}" for rep, ingr,alle in zip(recipe, ingredients,allergy)]
        for i, item in enumerate(texts):
            embeddings = self.embedding_model.encode(item).tolist()
            self.collection.add(embeddings=[embeddings], documents=[item], ids=[str(i)])

    # # Method to search the ChromaDB collection for relevant context based on a query
    def search_context(self, query, n_results=1):
        query_embeddings = self.embedding_model.encode(query).tolist()
        return self.collection.query(query_embeddings=query_embeddings, n_results=n_results)

In [3]:
# importing dataset hosted on huggingface
# dataset details - https://huggingface.co/datasets/Thefoodprocessor/recipe_new_with_features_full
dataset = load_dataset('Thefoodprocessor/recipe_new_with_features_full')

# create a vector embedding
vector_store = VectorStore("embedding_vector")
vector_store.populate_vectors(dataset)

In [4]:
# loading gpt4all language model
# load model Chat based model mistral-7b-openorca.gguf2.Q4_0.gguf
# detail about gpt4all and model information - https://gpt4all.io/index.html
model_name = 'mistral-7b-openorca.gguf2.Q4_0.gguf' 
model_path = Path.home() / '.cache' / 'gpt4all'
model = GPT4All(model_name=model_name, model_path=model_path)

In [8]:
conversation_history = []
# Define the chatbot response function connecting the history
def chatbot_response(user_input):
    global conversation_history
    results = vector_store.search_context(user_input, n_results=1)
    context = results['documents'][0] if results['documents'] else ""
    conversation_history.append(f"User: {user_input}\nContext: {context}\nBot:")
    response = model.generate("\n".join(conversation_history),temp=0)
    conversation_history.append(response)
    return response

In [9]:
def chat():
    print("What's cooking! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Ending the conversation. Happy meal!")
            break
        response = chatbot_response(user_input)
        print(f"Diet_chatbot: {response}")

In [10]:
chat()

What's cooking! Type 'exit' to end the conversation.
Diet_chatbot:  I understand that you have some carrots and rice. It seems like you are looking for a recipe to use these ingredients in. Here's a suggestion:

Ingredients:
- 1 cup of cooked brown or white rice (substitute with any other grain if desired)
- 2 medium carrots, peeled and grated
- 3 cloves garlic, minced
- 1/4 cup chopped fresh cilantro or parsley (optional)
- 1 tablespoon sesame oil or olive oil
- 2 teaspoons soy sauce or tamari
- Salt and pepper to taste

Instructions:
1. Cook the rice according to package instructions, then set aside.
2. In a large skillet or wok, heat the oil over medium heat. Add the garlic and cook for about 30 seconds until fragrant but not browned.
3. St
Diet_chatbot:  I understand that you have some milk. Here's a dessert recipe suggestion for you:

Ingredients:
- 1 cup whole milk (substitute with any other type of milk if desired)
- 2 tablespoons sugar
- 1/4 teaspoon vanilla extract
- Fresh fru